<a href="https://colab.research.google.com/github/tsunamizou/UKStockScreener/blob/main/ukstockscreener_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.oauth2.service_account import Credentials
import gspread
from google.colab import userdata
import json
from datetime import datetime
import yfinance as yf
import pandas as pd
import numpy as np


# ---------------- CONFIG ----------------
TICKERS = [
    'AZN.L','HSBA.L','SHEL.L','VOD.L','BP.L','GSK.L','LLOY.L','BARC.L',
    'RIO.L','ULVR.L','TSCO.L','DGE.L','BTI.L','PRU.L','CNA.L'
]
SPREADSHEET_NAME = 'UK Stock Screener'
SHEET_RANKING = 'Stock Ranking'
SHEET_TRANSACTIONS = 'Transactions'
SHEET_HOLDINGS = 'Portfolio Holdings'
SHEET_MONTHLY = 'Monthly Change'
SHEET_HISTORY = 'Ranking History'

TOTAL_PORTFOLIO = 10000  # GBP (for suggested allocation)
TOP_N = 15               # target portfolio size
SELL_RANK = 21           # sell when rank >= SELL_RANK
CONSECUTIVE_MONTHS_TO_SELL = 2

WEIGHTS = {
    '6m_return': 0.25,
    '3m_return': 0.15,
    'oper_margin': 0.20,
    'fcf_yield': 0.15,
    'dividend_yield': 0.10,
    'de_ratio': 0.15
}

# 3‑month return / 6‑month return: recent price momentum.
# Operating margin: profitability — helps avoid weak companies.
# Free Cash Flow (FCF) yield: cash‑generation ability.
# Debt/Equity: leverage — risk indicator.


In [2]:
# Load JSON from Colab Secrets
sa_info = json.loads(userdata.get('GoogleServiceAccount'))
SCOPES = ["https://www.googleapis.com/auth/spreadsheets",
          "https://www.googleapis.com/auth/drive"]

creds = Credentials.from_service_account_info(sa_info, scopes=SCOPES)

# load the google spreadsheet and individual sheets
gc = gspread.authorize(creds)
spreadsheet = gc.open(SPREADSHEET_NAME)


def get_sheet(name):
    try:
        return spreadsheet.worksheet(name)
    except gspread.exceptions.WorksheetNotFound:
        return spreadsheet.add_worksheet(title=name, rows="200", cols="20")


sheet_rank = get_sheet(SHEET_RANKING)
sheet_tx = get_sheet(SHEET_TRANSACTIONS)
sheet_hold = get_sheet(SHEET_HOLDINGS)
sheet_monthly = get_sheet(SHEET_MONTHLY)
sheet_history = get_sheet(SHEET_HISTORY)

# ---------- Helper: safe update to Google Sheets ----------
def safe_update(sheet, df):
    """Replace NaN/inf with empty strings, convert everything to Python native types and update sheet."""
    df2 = df.copy()
    # Replace infinite with NaN, then NaN -> ""
    df2 = df2.replace([np.inf, -np.inf], np.nan).fillna("")
    # Convert datetimes and numpy types to python types
    values = df2.astype(object).where(pd.notnull(df2), None).values.tolist()
    header = df2.columns.tolist()
    sheet.clear()
    sheet.update([header] + values)


In [3]:
# ---------- Fetch price history and compute returns ----------
def fetch_price_df(tickers, period="1y"):
    price_hist = {}
    for t in tickers:
        try:
            hist = yf.Ticker(t).history(period=period)
            if 'Close' in hist.columns and not hist['Close'].empty:
                price_hist[t] = hist['Close']
        except Exception as e:
            print(f"price fetch error {t}: {e}")
    price_df = pd.DataFrame(price_hist)
    return price_df

price_df = fetch_price_df(TICKERS)

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: BTI.L"}}}
ERROR:yfinance:$BTI.L: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


In [4]:
# ---------- Build features safely ----------
rows = []
for t in TICKERS:
    try:
        series = price_df.get(t)
        if series is None:
            continue
        prices = series.dropna()
        if len(prices) < 130:
            continue

        # returns (use iloc to avoid pandas warnings)
        ret_6m = (prices.iloc[-1] / prices.iloc[-126] - 1) * 100
        ret_3m = (prices.iloc[-1] / prices.iloc[-63] - 1) * 100

        stk = yf.Ticker(t)
        info = {}
        try:
            info = stk.info or {}
        except Exception:
            info = {}

        # Operating margin: many tickers may lack labels; we try safe extraction
        oper_margin = np.nan
        try:
            income = stk.financials
            # safe indexing with .iloc
            if 'Operating Income' in income.index and 'Total Revenue' in income.index:
                op = income.loc['Operating Income'].iloc[0]
                rev = income.loc['Total Revenue'].iloc[0]
                oper_margin = (op / rev) * 100 if (rev and rev != 0) else np.nan
            else:
                # banks / finance have different line items (e.g., 'Net Interest Income' or 'Net income')
                # attempt best-effort fallbacks
                possible_op = None
                for alt in ['Operating Income', 'OperatingIncome', 'OperatingProfit', 'Ebit', 'EBIT', 'EBITDA', 'Net Income']:
                    if alt in income.index:
                        possible_op = income.loc[alt].iloc[0]
                        break
                if possible_op is not None and 'Total Revenue' in income.index:
                    rev = income.loc['Total Revenue'].iloc[0]
                    oper_margin = (possible_op / rev) * 100 if (rev and rev != 0) else np.nan
        except Exception:
            oper_margin = np.nan

        # Free cash flow yield
        fcf_yield = np.nan
        try:
            cf = stk.cashflow
            if 'Free Cash Flow' in cf.index:
                fcf = cf.loc['Free Cash Flow'].iloc[0]
            else:
                # Try common alternate names
                fcf = None
                for alt in ['Free Cash Flow', 'FreeCashFlow', 'Cash Flow', 'Net Cash Provided by Operating Activities']:
                    if alt in cf.index:
                        fcf = cf.loc[alt].iloc[0]; break
            mc = info.get('marketCap', np.nan)
            if fcf is not None and mc and mc != 0:
                fcf_yield = (fcf / mc) * 100
        except Exception:
            fcf_yield = np.nan

        div = info.get('dividendYield', 0)

        # Only multiply if clearly fraction (<0.2 i.e., 20%)
        if div is None:
            dividend_yield = 0
        elif div <= 0.2:   # assume fraction < 20%
            dividend_yield = div * 100
        else:
            dividend_yield = div  # already in percent units
        de_ratio = info.get('debtToEquity', np.nan)
        sector = info.get('sector', 'Unknown')

        rows.append({
            'Ticker': t,
            '3m_return': ret_3m,
            '6m_return': ret_6m,
            'oper_margin': oper_margin,
            'fcf_yield': fcf_yield,
            'dividend_yield': dividend_yield,
            'de_ratio': de_ratio,
            'sector': sector
        })
    except Exception as e:
        print(f"skip {t} due to {e}")

df = pd.DataFrame(rows).set_index('Ticker')

In [5]:
df.head()

,3m_return,6m_return,oper_margin,fcf_yield,dividend_yield,de_ratio,sector
Ticker,,,,,,,
AZN.L,18.451025,26.814270,18.957705,3.470925,1.80,71.047,Healthcare
HSBA.L,7.182700,25.788590,35.579263,32.949245,4.68,NaN,Financial Services
SHEL.L,4.102508,2.685889,10.548974,22.460290,3.91,41.602,Energy
VOD.L,7.178962,24.770380,11.049989,39.107380,4.18,95.225,Communication Services
BP.L,6.127932,14.328487,7.196659,17.622324,5.43,96.358,Energy


In [6]:
# ---------- Normalize / rank features safely ----------
# We'll rank each column to [0,1] (higher = better). For de_ratio, lower is better so invert it.
rank_df = pd.DataFrame(index=df.index)

# Utility to safe-rank a column (returns 0..1), if entire col NaN returns 0.5
def safe_rank(series, invert=False):
    if series.dropna().empty:
        return pd.Series(0.5, index=series.index)  # neutral
    # If invert=True, lower actual value is better -> we rank ascending=False to give lower value higher score
    ranked = series.rank(pct=True, ascending=not invert)
    # Convert to numeric and keep missing as NaN for now
    return ranked

rank_df['6m_return'] = safe_rank(df['6m_return'])
rank_df['3m_return'] = safe_rank(df['3m_return'])
rank_df['oper_margin'] = safe_rank(df['oper_margin'])
rank_df['fcf_yield'] = safe_rank(df['fcf_yield'])
rank_df['dividend_yield'] = safe_rank(df['dividend_yield'])
# For debt-to-equity: lower is better => invert=True
rank_df['de_ratio'] = safe_rank(df['de_ratio'], invert=True)

# Fill missing ranks with neutral 0.5 (Option A)
rank_df = rank_df.fillna(0.5)

# Compute weighted TotalScore
rank_df['TotalScore'] = 0
for col, w in WEIGHTS.items():
    if col not in rank_df.columns:
        continue
    rank_df['TotalScore'] += rank_df[col] * w

# Rank by TotalScore (1 best)
rank_df['Rank'] = rank_df['TotalScore'].rank(ascending=False, method='min')

# Add raw fundamentals/prices for context to Ranking sheet
# 1️⃣ Copy ranked DataFrame
out_df = rank_df.copy()

# 2️⃣ Add raw metrics with '_raw' suffix (to avoid conflicts)
raw_cols = ['6m_return', '3m_return', 'oper_margin', 'fcf_yield', 'dividend_yield', 'de_ratio', 'sector']
out_df = out_df.join(df[raw_cols].add_suffix('_raw'), how='left')

# Take the last row (latest date) of price_df
current_prices = price_df.tail(1).T.iloc[:, 0]  # .iloc[:,0] grabs the only column
# Convert to dict for mapping to out_df
out_df['Price'] = out_df.index.map(current_prices.to_dict())

# 3️⃣ Sort by Rank (ascending: 1 = best)
out_df = out_df.sort_values('Rank')



In [7]:
# ---------- Write ranking to Google Sheets ----------
safe_update(sheet_rank, out_df.reset_index())

# 2️⃣ Prepare snapshot for this month
month = datetime.now().strftime('%Y-%m-%d')
snapshot = out_df.reset_index().copy()
snapshot.insert(0, 'Date', month)
# Optional: only keep relevant columns
cols_to_save = ['Date', 'Ticker', 'Rank', 'TotalScore','Price',
                '6m_return_raw','3m_return_raw','oper_margin_raw','fcf_yield_raw','dividend_yield_raw','de_ratio_raw']
snapshot = snapshot[cols_to_save]

# 3️⃣ Append to Google Sheet
existing = sheet_history.get_all_records()
existing_df = pd.DataFrame(existing)
if existing_df.empty:
    safe_update(sheet_history, snapshot)
else:
    combined = pd.concat([existing_df, snapshot], ignore_index=True)
    safe_update(sheet_history, combined)

In [8]:
# ---------- Monthly change tracking & simple buy/sell signals ---------


# ---------- Load current holdings ----------
ph_rows = sheet_hold.get_all_records()
ph_df = pd.DataFrame(ph_rows)

# If your sheet uses "Ticker" column
if not ph_df.empty and "Ticker" in ph_df.columns:
    holdings = set(ph_df["Ticker"].dropna().astype(str).unique())
else:
    holdings = set()

# ---------- Load previous MonthlyChange ----------
prev_records = sheet_monthly.get_all_records()
prev_df = pd.DataFrame(prev_records)

# Current month
month = datetime.now().strftime('%Y-%m-%d')

mc_rows = []

# Precompute previous prices if available
prev_prices = {}
if not prev_df.empty and "Ticker" in prev_df.columns and "CurrentPrice" in prev_df.columns:
    # Take the last known current price for each ticker as previous price
    last_entries = prev_df.groupby("Ticker").last()
    for t in last_entries.index:
        prev_prices[t] = last_entries.at[t, "CurrentPrice"]

for t in out_df.index:
    rank_now = int(out_df.at[t, 'Rank']) if pd.notnull(out_df.at[t, 'Rank']) else None
    curr_price = float(out_df.at[t, 'Price']) if 'Price' in out_df.columns and pd.notnull(out_df.at[t, 'Price']) else None
    prev_rank = ''
    prev_price = prev_prices.get(t, '')

    # Determine previous rank if available
    if not prev_df.empty and 'Ticker' in prev_df.columns:
        match = prev_df[prev_df['Ticker'] == t]
        if not match.empty:
            prev_rank_list = match['Rank'].dropna().astype(int).tolist()
            if prev_rank_list:
                prev_rank = prev_rank_list[-1]

    # Default signal
    signal = 'HOLD'

    # BUY: enters TOP_N and not currently held
    if rank_now is not None and rank_now <= TOP_N and t not in holdings:
        signal = 'BUY'

    # SELL: held and rank >= SELL_RANK for CONSECUTIVE_MONTHS_TO_SELL
    if t in holdings:
        last_ranks = []
        if not prev_df.empty and t in prev_df['Ticker'].values:
            prev_entries = prev_df[prev_df['Ticker'] == t]
            prev_ranks_list = prev_entries['Rank'].dropna().astype(int).tolist()
            if CONSECUTIVE_MONTHS_TO_SELL > 1:
                last_ranks = prev_ranks_list[-(CONSECUTIVE_MONTHS_TO_SELL - 1):] if prev_ranks_list else []
        combined = last_ranks + ([rank_now] if rank_now is not None else [])
        if len(combined) >= CONSECUTIVE_MONTHS_TO_SELL and all(r >= SELL_RANK for r in combined):
            signal = 'SELL'

    mc_rows.append({
        'Date': month,
        'Ticker': t,
        'PrevRank': prev_rank,
        'Rank': rank_now if rank_now is not None else '',
        'PrevPrice': prev_price,
        'CurrentPrice': curr_price,
        'Signal': signal
    })

# Build new DataFrame
mc_df_new = pd.DataFrame(mc_rows)

# Append to existing MonthlyChange sheet
if prev_df.empty:
    safe_update(sheet_monthly, mc_df_new)
else:
    combined_df = pd.concat([prev_df, mc_df_new], ignore_index=True)
    safe_update(sheet_monthly, combined_df)

print(f"MonthlyChange updated for {month}: {len(mc_rows)} tickers.")

MonthlyChange updated for 2025-12-10: 14 tickers.
